<a href="https://colab.research.google.com/github/yudumpacin/LLM/blob/main/RAG_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain_community pypdf sentence-transformers faiss-gpu peft bitsandbytes accelerate --quiet

In [2]:
!wget "https://www.tbb.org.tr/Content/Upload/Dokuman/836/dergi_82.pdf"

--2024-05-22 21:01:47--  https://www.tbb.org.tr/Content/Upload/Dokuman/836/dergi_82.pdf
Resolving www.tbb.org.tr (www.tbb.org.tr)... 62.244.232.194
Connecting to www.tbb.org.tr (www.tbb.org.tr)|62.244.232.194|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1457994 (1.4M) [application/pdf]
Saving to: ‘dergi_82.pdf.1’

dergi_82.pdf.1      100%[===================>]   1.39M   243KB/s    in 7.3s    

2024-05-22 21:01:56 (195 KB/s) - ‘dergi_82.pdf.1’ saved [1457994/1457994]



In [3]:
from langchain_community.document_loaders import TextLoader, PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter

loader = PyPDFLoader("dergi_82.pdf")

documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
texts = text_splitter.split_documents(documents)
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [34]:
vectorstore = FAISS.from_documents(texts, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={'k': 1})

In [5]:
query="FATF terminolojisinde “Finansal Olmayan Belirli iş ve Meslekler nelerdir?"

In [6]:
from langchain.chains import RetrievalQA
from langchain_core.prompts import PromptTemplate

In [38]:
template_english = """
Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:
"""

In [39]:
template_turkish = """
Verilen dokümanı kullanarak, sorulan soruya cevap ver.
Sorunun cevabını bilmiyorsan, Bilmiyorum yaz.
Doküman: {context}
Soru: {question}
Cevap:
"""

In [40]:
PROMPT = PromptTemplate(input_variables=["context", "question"], template=template_turkish)

In [14]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, pipeline
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

In [10]:
lora_model_id="Yudum/llama3-lora-turkish"

In [11]:
model = AutoPeftModelForCausalLM.from_pretrained(
        lora_model_id,
        load_in_4bit = True,
    )
tokenizer = AutoTokenizer.from_pretrained(lora_model_id)
model = model.merge_and_unload()
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer,max_new_tokens=128)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:167: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:325: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [15]:
hf = HuggingFacePipeline(pipeline=pipe)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the from rom langchain-huggingface package and should be used instead. To use it run `pip install -U from rom langchain-huggingface` and import as `from from rom langchain_huggingface import llms import HuggingFacePipeline`.
  warn_deprecated(


In [17]:
hf(query)

'FATF terminolojisinde “Finansal Olmayan Belirli iş ve Meslekler nelerdir?”, sorusu sorulmuştur. Bu sorunun cevabı şu şekildedir: “Belirli iş ve meslekler, belirli bir sektörün veya belirli bir finansal hizmetin veya finansal aracının veya finansal araçların kullanımının finansal işlem veya finansal faaliyetin niteliğini belirleyen faktörleridir. Bu faktörler, bir finansal işlem veya finansal faaliyetin finansal işlem veya finansal faaliyet olarak kabul edilmesi için gerekli olabilecek bir veya daha fazla finansal hizmet veya finansal araçın kullanımını içerir.”[3]\n\nBu terim, 2008 Finans'

In [41]:
chain = RetrievalQA.from_chain_type(
    llm=hf,
    chain_type='stuff',
    retriever=retriever,
    input_key='query',
    return_source_documents=True,
    chain_type_kwargs={"prompt":PROMPT},
    verbose=True
)

In [42]:
result = chain(query)



> Entering new RetrievalQA chain...

> Finished chain.


In [45]:
print(result["result"])


Verilen dokümanı kullanarak, sorulan soruya cevap ver. 
Sorunun cevabını bilmiyorsan, Bilmiyorum yaz.  
Doküman: hesaplar ÕnÕn paravan bankalar taraf Õndan kullan ÕlmasÕna izin verilmedi ÷inden emin olmalar Õ 
zorunlulu ÷u getirilmelidir. 
 
1.12.2. Yeni Teknolojilerden Kaynaklanan Risklere Dikkat Edilmesi  
 Finansal kurulu úlar, yeni ürünlerin ve i ú uygulamalar ÕnÕn geliúmesine dikkat etmeli ve 
bu ürünler için yeni geli úen teknolojilerin kullan ÕmÕna ba÷lÕ olarak ortaya ç Õkabilecek karapara 
aklama ve terörün finansman Õ risklerini tespit etmeli ve de ÷erlendirmelidir.  
 
2. Finansal Olmayan Belirli øú ve Mesleklere Ait Yükümlülükler  
2.1. Finansal Olmayan Belirli øú ve Meslekler  
  
FATF terminolojisinde “Finansal Olmayan Belirli øú ve Meslekler 
 
a) Kumarhaneler( internet kumarhaneleri dahil), 
b) Emlakç Õlar, 
c) KÕymetli maden tacirleri, 
d) KÕymetli taú tacirleri, 
e) Avukatlar, noterler ve muhasebeciler,  f) Trustlar
1 ve úirket hizmeti sa ÷layanlar 
 
olarak belirlenm